In [1]:
import time
import pickle
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier 
import matplotlib.pyplot as plt 
import seaborn as sns 
import pymongo
from pymongo import MongoClient

In [3]:
conection=MongoClient("mongodb://localhost:27017/")
db=conection.Iris2
data=db.iris2
df = pd.DataFrame(list(data.find()))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df[['SepalLengthCm', 'SepalWidthCm','PetalLengthCm', 'PetalWidthCm']],df['Species'], random_state=0) 

In [5]:
print("X_train shape: {}\ny_train shape: {}".format(X_train.shape, y_train.shape))
print("X_test shape: {}\ny_test shape: {}".format(X_test.shape, y_test.shape))

X_train shape: (112, 4)
y_train shape: (112,)
X_test shape: (38, 4)
y_test shape: (38,)


In [6]:
# initialize the Estimator object
knn = KNeighborsClassifier(n_neighbors=1)

In [7]:
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [8]:
pd.concat([X_test, y_test, pd.Series(y_pred, name='Predicted', index=X_test.index)],ignore_index=False, axis=1)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Predicted
114,5.8,2.8,5.1,2.4,Iris-virginica,Iris-virginica
62,6.0,2.2,4.0,1.0,Iris-versicolor,Iris-versicolor
33,5.5,4.2,1.4,0.2,Iris-setosa,Iris-setosa
107,7.3,2.9,6.3,1.8,Iris-virginica,Iris-virginica
7,5.0,3.4,1.5,0.2,Iris-setosa,Iris-setosa
100,6.3,3.3,6.0,2.5,Iris-virginica,Iris-virginica
40,5.0,3.5,1.3,0.3,Iris-setosa,Iris-setosa
86,6.7,3.1,4.7,1.5,Iris-versicolor,Iris-versicolor
76,6.8,2.8,4.8,1.4,Iris-versicolor,Iris-versicolor
71,6.1,2.8,4.0,1.3,Iris-versicolor,Iris-versicolor


In [9]:
print("Test set score: {:.2f}".format(knn.score(X_test, y_test)))


Test set score: 0.97


In [10]:
model = knn

In [11]:
filename = 'final.sav'
pickle.dump(model, open(filename, 'wb'))

In [12]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9736842105263158


In [13]:
def save_model_to_db(model,client,db,dbconnection,model_name):
    pickled_model=pickle.dumps(model)
    # creating connection
    myclient=pymongo.MongoClient(client)
    #creating database in mongodb mydb myclient[db)
    mydb=myclient[db]
    #creating collection
    mycon=mydb[dbconnection]
    info=mycon.insert_one({model_name: pickled_model, 'name': model_name, 'created_time':time.time()})
    print(info.inserted_id,"saved with this id successfullyt")
    details={
        'inserted_id':info.inserted_id,
        "model_name":model_name,
        'created_time':time.time()
    }
    return details

In [17]:
details=save_model_to_db(model=knn,client='mongodb://localhost:27017/',db='Iris2',dbconnection='data1',model_name='final1')


629a20062f3cc0ca70b00584 saved with this id successfullyt


In [15]:
def load_saved_model_from_db(model_name,client,db,dbconnection):
    json_data={}
    myclient=pymongo.MongoClient(client)
    #creating database in mongodb mydb myclient[db)
    mydb=myclient[db]
    #creating collection
    mycon=mydb[dbconnection]
    data=mycon.find({'name':model_name})
    
    for i in data:
        json_data=i
    #fetching model from_db
    pickled_model=json_data[model_name]
    return pickle.loads(pickled_model)
    

In [22]:
details1=load_saved_model_from_db(model_name='final1',client='mongodb://localhost:27017/',db='Iris2',dbconnection='data1')

In [24]:
print(details1.score(X_test, y_test))

0.9736842105263158
